In [0]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
import pickle

In [2]:
!wget https://github.com/dating-success-app/Data-Science/raw/master/data_exploration/Data/profiles.csv.zip
!unzip profiles.csv.zip

--2019-07-31 07:27:50--  https://github.com/dating-success-app/Data-Science/raw/master/data_exploration/Data/profiles.csv.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dating-success-app/Data-Science/master/data_exploration/Data/profiles.csv.zip [following]
--2019-07-31 07:27:50--  https://raw.githubusercontent.com/dating-success-app/Data-Science/master/data_exploration/Data/profiles.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56227692 (54M) [application/zip]
Saving to: ‘profiles.csv.zip’

profiles.csv.zip    100%[===================>]  53.62M  94.2MB/s    in 0.6s    

2019-07-31 07

In [3]:
df = pd.read_csv('./profiles.csv')

print(df.shape,'\n',
      df.columns,'\n')
df.head()

(59946, 31) 
 Index(['age', 'body_type', 'diet', 'drinks', 'drugs', 'education', 'essay0',
       'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7',
       'essay8', 'essay9', 'ethnicity', 'height', 'income', 'job',
       'last_online', 'location', 'offspring', 'orientation', 'pets',
       'religion', 'sex', 'sign', 'smokes', 'speaks', 'status'],
      dtype='object') 



,age,body_type,diet,drinks,drugs,education,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9,ethnicity,height,income,job,last_online,location,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status
0,22,a little extra,strictly anything,socially,never,working on college/university,about me:<br />\n<br />\ni would love to think...,currently working as an international agent fo...,making people laugh.<br />\nranting about a go...,"the way i look. i am a six foot half asian, ha...","books:<br />\nabsurdistan, the republic, of mi...",food.<br />\nwater.<br />\ncell phone.<br />\n...,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet!<br />\nyou...,"asian, white",75.0,-1,transportation,2012-06-28-20-30,"south san francisco, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism and very serious about it,m,gemini,sometimes,english,single
1,35,average,mostly other,often,sometimes,working on space camp,i am a chef: this is what that means.<br />\n1...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories.<br /...,NaN,NaN,i am very open and will share just about anyth...,NaN,white,70.0,80000,hospitality / travel,2012-06-29-21-41,"oakland, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism but not too serious about it,m,cancer,no,"english (fluently), spanish (poorly), french (...",single
2,38,thin,anything,socially,NaN,graduated from masters program,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement<br />\nconversation<br />\ncreation<b...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ...",NaN,68.0,-1,NaN,2012-06-27-09-10,"san francisco, california",NaN,straight,has cats,NaN,m,pisces but it doesn&rsquo;t matter,no,"english, french, c++",available
3,23,thin,vegetarian,socially,NaN,working on college/university,i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . .<br />\nlynch, j...",NaN,cats and german philosophy,NaN,NaN,you feel so inclined.,white,71.0,20000,student,2012-06-28-14-22,"berkeley, california",doesn&rsquo;t want kids,straight,likes cats,NaN,m,pisces,no,"english, german (poorly)",single
4,29,athletic,NaN,socially,never,graduated from college/university,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at:<br />\nhttp://bag...,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians<br />\nat the...",NaN,NaN,NaN,NaN,NaN,"asian, black, other",66.0,-1,artistic / musical / writer,2012-06-27-21-26,"san francisco, california",NaN,straight,likes dogs and likes cats,NaN,m,aquarius,no,english,single


In [6]:
for i in np.arange(30):
    
    print(df['essay2'][i],'\n')

making people laugh.<br />
ranting about a good salting.<br />
finding simplicity in complexity, and complexity in simplicity. 

being silly. having ridiculous amonts of fun wherever. being a
smart ass. ohh and i can cook. ;) 

improvising in different contexts. alternating between being
present and decidedly outside of a moment, or trying to hold both
at once. rambling intellectual conversations that hold said
conversations in contempt while seeking to find something that
transcends them. being critical while remaining generous. listening
to and using body language--often performed in caricature or large
gestures, if not outright interpretive dance. dry, dark, and
raunchy humor. 

playing synthesizers and organizing books according to the library
of congress classification system 

creating imagery to look at:<br />
http://bagsbrown.blogspot.com/<br />
http://stayruly.blogspot.com/ 

imagining random shit. laughing at aforementioned random shit.
being goofy. articulating what i think 

In [0]:
desc = df['essay2'].dropna().values

In [8]:
len(desc)

50308

In [0]:
stemmer = SnowballStemmer('english')
tokenizer = RegexpTokenizer(r'[a-zA-Z\']+')

def tokenize(text):
    return [stemmer.stem(word) for word in tokenizer.tokenize(text.lower())]

In [0]:
punc = ['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}',"%"]
ex_stop = ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 
           'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 
           'besid', 'br', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 
           'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 
           'henc', 'hereaft', 'herebi', 'howev', 'href', 'hundr', 'inde', 'mani', 
           'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 
           'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 
           'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 
           'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 
           'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 
           'wherev', 'whi', 'yourselv','anywh', 'el', 'elsewh', 'everywh', 'ind', 
           'otherwi', 'plea', 'somewh']

stop_words = text.ENGLISH_STOP_WORDS.union(ex_stop).union(punc)
vectorizer = TfidfVectorizer(stop_words = stop_words, tokenizer=tokenize, max_features=6000)
X = vectorizer.fit_transform(desc)

In [7]:
vectorizer.get_feature_names()

["'s",
 'ab',
 'abalon',
 'abandon',
 'abil',
 'abl',
 'abroad',
 'absolut',
 'absorb',
 'abstract',
 'absurd',
 'abund',
 'abus',
 'academ',
 'academia',
 'acceler',
 'accent',
 'accept',
 'access',
 'accessor',
 'accessori',
 'accid',
 'accident',
 'accommod',
 'accompani',
 'accomplish',
 'accord',
 'accordion',
 'account',
 'accumul',
 'accur',
 'accuraci',
 'accus',
 'ace',
 'ach',
 'achiev',
 'acknowledg',
 'acoust',
 'acquaint',
 'acquir',
 'acquisit',
 'acrobat',
 'acronym',
 'acryl',
 'act',
 'action',
 'activ',
 'activist',
 'actor',
 'actress',
 'actual',
 'acut',
 'ad',
 'adam',
 'adapt',
 'add',
 'addict',
 'addit',
 'address',
 'adept',
 'adequ',
 'adhd',
 'adject',
 'adjust',
 'administ',
 'administr',
 'admir',
 'admit',
 'adob',
 'adobo',
 'adolesc',
 'adopt',
 'ador',
 'adrenalin',
 'adult',
 'advanc',
 'advantag',
 'adventur',
 'advers',
 'advertis',
 'advic',
 'advis',
 'advoc',
 'advocaci',
 'aerial',
 'aerob',
 'aesthet',
 'affair',
 'affect',
 'affection',
 'affi

In [0]:
df['location'].value_counts()

san francisco, california       31064
oakland, california              7214
berkeley, california             4212
san mateo, california            1331
palo alto, california            1064
                                ...  
orange, california                  1
hacienda heights, california        1
waterford, california               1
union city, california              1
astoria, new york                   1
Name: location, Length: 199, dtype: int64

In [8]:
kmeans = KMeans(n_clusters = 10, n_init = 5, n_jobs = -1)
kmeans.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=10, n_init=5, n_jobs=-1, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [0]:
words = vectorizer.get_feature_names()

In [11]:
common_words = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

0 : listen, friend, good, talk, peopl, laugh, advic, cook, problem, danc
1 : laugh, make, peopl, good, cook, smile, listen, feel, friend, time
2 : peopl, make, laugh, feel, smile, good, friend, comfort, listen, new
3 : make, play, listen, friend, good, danc, write, time, like, peopl
4 : cook, love, danc, listen, friend, make, good, bake, laugh, thing
5 : good, i'm, realli, pretti, peopl, thing, friend, like, make, think
6 : thing, fix, lot, new, make, tri, good, learn, peopl, listen
7 : ilink, class, cook, i'm, danc, good, make, thing, peopl, game
8 : parti, dinner, throw, make, host, peopl, friend, plan, cook, danc
9 : life, enjoy, live, peopl, good, make, laugh, thing, friend, fun


In [16]:
kmeans.cluster_centers_

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.43769794e-04, 1.00838990e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 5.11464947e-05],
       ...,
       [9.13857600e-04, 0.00000000e+00, 0.00000000e+00, ...,
        2.21130582e-04, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        3.81855425e-04, 0.00000000e+00, 9.86560355e-04],
       [1.96986383e-04, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 2.82954756e-04]])

In [0]:
new_string = ['cook love dance listen friend bake laugh']

In [0]:
Y = vectorizer.transform(new_string)

In [27]:
kmeans.predict(Y)

array([4], dtype=int32)

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(kmeans.cluster_centers_[4].reshape(1,-1), Y.toarray().reshape(1,-1)) 

array([[0.50975899]])

In [0]:

filename = 'means.sav'
pickle.dump(kmeans, open(filename, 'wb'))

In [0]:
new_vect = pickle.load(open(filename, 'rb'))

AttributeError: Can't get attribute 'tokenize' on <module '__main__'>

In [0]:
vectorizer.transform(new_string)

<1x6000 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [0]:
(new_vect.transform(new_string).todense()).shape

(1, 6000)

In [0]:
sf = (df[df['location'] == 'san francisco, california'][['essay2']]
      .dropna()
      .reset_index()
      .drop(columns=['index']))

In [0]:
sf.to_csv('sf_data_descriptions.csv')

In [0]:
tokenize(new_string[0])

['i', 'love', 'go', 'out', 'for', 'taco', 'hike', 'and', 'craft', 'cocktail']

In [0]:
len(str(new_string))

58